## **Этот ноутбук**: токенизирует текста и смотрит полезные токены, обрезает словарь, эмбеддинги модели, а так же оставляет в модели только те слои, что было решено оставить после анализа и сохраняет обновлённую модель.

# Прунинг словаря (10 языков)

In [ ]:
# # # 1. Монтируем Google Drive
from google.colab import drive
drive.mount('/content/drive')



# 2. Копируем kaggle.json в нужную директорию
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

# 3. Устанавливаем пра3ва на файл
!chmod 600 ~/.kaggle/kaggle.json
# 4. Устанавливаем kaggle API (если не установлен)
# !pip install -q kaggle

# 5. Пример скачивания датасета
dataset_name = 'kehhill/queries'

dataset_id = dataset_name.split('/')[-1]

# # 5. Скачиваем датасет
!kaggle datasets download -d {dataset_name}

# # 6. Создаём папку и распаковываем тудк1а
!mkdir -p {dataset_id}

!unzip -q "{dataset_id}.zip" -d {dataset_id}

Mounted at /content/drive
Dataset URL: https://www.kaggle.com/datasets/kehhill/queries
License(s): unknown
 98% 684M/702M [00:03<00:00, 135MB/s]
100% 702M/702M [00:03<00:00, 193MB/s]


In [ ]:
import pandas as pd
tf = pd.read_parquet("/content/queries/transef_lerning_merged_10_langueges.parquet")
fine_tine = pd.read_parquet("/content/queries/fine_tune_10_langueges.parquet")

In [ ]:
extended_texts = []

for i in range(len(tf)):
    for col in tf.columns:
        extended_texts.append(tf[col].iloc[i])

for i in range(len(fine_tine)):
    for col in fine_tine.columns:
        extended_texts.append(fine_tine[col].iloc[i])

In [ ]:
!pip install wordfreq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00


In [ ]:
from wordfreq import top_n_list
import pandas as pd
import random
import string

# ===== Настройки =====
languages = ["en", "es", "de", "fr", "pt", "ru", "it", "nl", "pl", "tr"]
top_n = 300_000
words_per_sentence = 20

# ===== Сбор слов =====
all_words = []
for lang in languages:
    words = top_n_list(lang, top_n)
    all_words.extend(words)

# ===== Перемешиваем и формируем "предложения" =====
random.shuffle(all_words)

sentences = []
for i in range(0, len(all_words), words_per_sentence):
    chunk = all_words[i:i+words_per_sentence]
    if len(chunk) == words_per_sentence:
        sentences.append(" " + " ".join(chunk) + " ")

# ===== Добавляем "спец предложения" с пунктуацией =====
special_chars = list(string.punctuation) + ["«", "»", "—", "…", "“", "”", "„", "’", "‘"]
special_sentence = " " + " ".join(special_chars) + " "
sentences.append(special_sentence)

# ===== Сохраняем в Parquet =====
df = pd.DataFrame({"text": sentences+extended_texts})
output_file = "multilang_vocab_dataset.parquet"
df.to_parquet(output_file, index=False)

# ===== Вывод информации =====
(df.shape, output_file)


((1725396, 1), 'multilang_vocab_dataset.parquet')

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
import pandas as pd
from tqdm import tqdm
import json
import os

# ==== Настройки ====
MODEL_NAME = "BAAI/bge-m3"
INPUT_PARQUET = "multilang_vocab_dataset.parquet"  # твой датасет
SAVE_PATH = "bge-m3-reduced"

# ==== Загрузка токенайзера и модели ====
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# ==== 1. Сбор used_token_ids ====
df = pd.read_parquet(INPUT_PARQUET)

unique_token_ids = set()
for text in tqdm(df["text"], desc="Tokenizing", ncols=100, dynamic_ncols=True, leave=True):
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    unique_token_ids.update(token_ids)

with open(f"unique_tokens_ids.txt", "w", encoding="utf-8") as f:
    for tid in sorted(unique_token_ids):
        f.write(f"{tid}\n")



Tokenizing: 100%|██████████| 1725396/1725396 [05:53<00:00, 4886.42it/s]


# Прунинг модели


In [ ]:
import os
import json
from transformers import AutoTokenizer, AutoModel

# === Настройки ===
MODEL_NAME = "BAAI/bge-m3"
USED_TOKEN_IDS_PATH = "/content/unique_tokens_ids.txt"


# === Загрузка токенизатора и ID из датасета ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# загружаем модель
model = AutoModel.from_pretrained(MODEL_NAME)

with open(USED_TOKEN_IDS_PATH, "r", encoding="utf-8") as f:
    unique_token_ids = sorted([
        int(line.strip()) for line in f
        if line.strip().isdigit()
    ])

old_vocab = tokenizer.get_vocab()
core_spec = ['<s>', '</s>', '<unk>', '<pad>']
id_spec_tok = {t: old_vocab[t] for t in tokenizer.all_special_tokens if t in core_spec}
additional = [old_vocab[t] for t in tokenizer.all_special_tokens if t not in core_spec]


all_ids = sorted([i for i in list(set(unique_token_ids)) if i not in id_spec_tok.values()]+additional)
start_id = 4
switch_vocab = {k: v for v, k in old_vocab.items()}
id_to_token = {switch_vocab[id]: i+start_id for i, id in zip(range(len(all_ids)), all_ids)}
filtered_vocab = {k: v for k, v in list(id_spec_tok.items())+list(id_to_token.items())}


In [ ]:
from transformers import AutoTokenizer, PreTrainedTokenizerFast
from tokenizers import Tokenizer
from tokenizers.models import Unigram

In [ ]:
backend_vocab_list = sorted(filtered_vocab.items(), key=lambda x: x[1])  # сортируем по ID
backend_model = Unigram(vocab=backend_vocab_list, unk_id=filtered_vocab.get(tokenizer.unk_token, 0))
backend_tokenizer = Tokenizer(backend_model)

# 4. Копируем нормализаторы, препроцессоры и постпроцессоры из старого токенайзера
backend_tokenizer.normalizer = tokenizer.backend_tokenizer.normalizer
backend_tokenizer.pre_tokenizer = tokenizer.backend_tokenizer.pre_tokenizer
backend_tokenizer.decoder = tokenizer.backend_tokenizer.decoder
backend_tokenizer.post_processor = tokenizer.backend_tokenizer.post_processor

# 5. Создаём новый PreTrainedTokenizerFast
new_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=backend_tokenizer,
    bos_token=tokenizer.bos_token,
    eos_token=tokenizer.eos_token,
    unk_token=tokenizer.unk_token,
    pad_token=tokenizer.pad_token,
    cls_token=tokenizer.cls_token,
    sep_token=tokenizer.sep_token,
    mask_token=tokenizer.mask_token
)

# 6. Проверяем
print(new_tokenizer.vocab_size)
print(new_tokenizer.tokenize("Привет мир"))
print(new_tokenizer.encode("Привет мир"))


88795
['▁Пр', 'и', 'вет', '▁м', 'ир']
[0, 25717, 86, 10903, 2380, 12472, 2]


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel


# 2. Получаем старые эмбеддинги
old_emb = model.get_input_embeddings().weight.data.clone()

# 3. Создаём новый тензор эмбеддингов
new_vocab_size = len(filtered_vocab)
embedding_dim = old_emb.size(1)
new_emb = torch.zeros((new_vocab_size, embedding_dim), dtype=old_emb.dtype)

# 4. Переносим веса для токенов, которые остались
for token, new_id in filtered_vocab.items():
    old_id = old_vocab[token]
    new_emb[new_id] = old_emb[old_id]



In [ ]:
# 5. Обновляем эмбеддинги модели
model.resize_token_embeddings(new_vocab_size)  # создаём новый слой нужного размера
model.get_input_embeddings().weight.data = new_emb
# 6. Обновляем конфиг
model.config.vocab_size = new_vocab_size

In [ ]:

# tokenizer.save_pretrained(small_SAVE_PATH)
import torch.nn as nn

cfg = model.config

print('ДО')
print(f"layers={cfg.num_hidden_layers}, ffn_dim={cfg.intermediate_size}, max_pos={cfg.max_position_embeddings}")
print(f"Параметров: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")
old_param = sum(p.numel() for p in model.parameters())
print('-'*20)
print()






# === Обновление конфига ===

import torch
from transformers import AutoModel, AutoTokenizer

MODEL_NAME = "BAAI/bge-m3"
KEEP_LAYERS = [1,2,5,9,13,17,20, 23,24]

# урезаем encoder
orig_layers = model.encoder.layer
new_layers = torch.nn.ModuleList([orig_layers[i-1] for i in KEEP_LAYERS])
model.encoder.layer = new_layers

# обновляем конфиг
model.config.num_hidden_layers = len(KEEP_LAYERS)

print(f"Оставлено {len(KEEP_LAYERS)} слоёв: {KEEP_LAYERS}")

print('ПОСЛЕ')
print(f"layers={cfg.num_hidden_layers}, ffn_dim={cfg.intermediate_size}, max_pos={cfg.max_position_embeddings}")
print(f"Параметров: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")
new_param = sum(p.numel() for p in model.parameters())

print('\nИтог:')
print(f"Количество параметром уменьшилось на {round(100*(1-new_param/old_param), 2)} %")


ДО
layers=24, ffn_dim=4096, max_pos=8194
Параметров: 402.7M
--------------------

Оставлено 9 слоёв: [1, 2, 5, 9, 13, 17, 20, 23, 24]
ПОСЛЕ
layers=9, ffn_dim=4096, max_pos=8194
Параметров: 213.7M

Итог:
Количество параметром уменьшилось на 46.92 %


## Сохраняем модель

In [ ]:
save_small = '/content/drive/MyDrive/bge_fast_ten_lang'
os.makedirs(save_small, exist_ok=True)
model.save_pretrained(save_small)
new_tokenizer.save_pretrained(save_small)

('/content/drive/MyDrive/bge_fast_ten_lang/tokenizer_config.json',
 '/content/drive/MyDrive/bge_fast_ten_lang/special_tokens_map.json',
 '/content/drive/MyDrive/bge_fast_ten_lang/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/bge_fast_ten_lang')
# загружаем модель
model = AutoModel.from_pretrained('/content/drive/MyDrive/bge_fast_ten_lang')

In [ ]:
sum(p.numel() for p in model.parameters())

213735424